In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import svm, linear_model, datasets
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.decomposition import PCA
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, ConstantKernel

In [2]:
df = pd.read_csv('data/ParisHousing.csv') #loads data

In [3]:
pd.set_option('display.precision', 2) #formatting for printed output
#remove binary features from summary
df.drop(columns = ['hasYard', 'hasPool', 'isNewBuilt', 'hasStormProtector',
                   'hasStorageRoom', 'hasGuestRoom']).describe().T

,count,mean,std,min,25%,50%,75%,max
squareMeters,10000.0,4.99e+04,2.88e+04,89.0,2.51e+04,5.01e+04,7.46e+04,1.00e+05
numberOfRooms,10000.0,5.04e+01,2.88e+01,1.0,2.50e+01,5.00e+01,7.50e+01,1.00e+02
floors,10000.0,5.03e+01,2.89e+01,1.0,2.50e+01,5.00e+01,7.60e+01,1.00e+02
cityCode,10000.0,5.02e+04,2.90e+04,3.0,2.47e+04,5.07e+04,7.57e+04,1.00e+05
cityPartRange,10000.0,5.51e+00,2.87e+00,1.0,3.00e+00,5.00e+00,8.00e+00,1.00e+01
numPrevOwners,10000.0,5.52e+00,2.86e+00,1.0,3.00e+00,5.00e+00,8.00e+00,1.00e+01
made,10000.0,2.01e+03,9.31e+00,1990.0,2.00e+03,2.01e+03,2.01e+03,2.02e+03
basement,10000.0,5.03e+03,2.88e+03,0.0,2.56e+03,5.09e+03,7.51e+03,1.00e+04
attic,10000.0,5.03e+03,2.89e+03,1.0,2.51e+03,5.04e+03,7.54e+03,1.00e+04
garage,10000.0,5.53e+02,2.62e+02,100.0,3.28e+02,5.54e+02,7.77e+02,1.00e+03


Classification on cityPartRange (1-10 value representing the desirability of a neighborhood)

In [ ]:
X = df.drop(columns='cityPartRange')
y = df.cityPartRange
# split 50% training data, 50% "_tmp" for validation & test
X_train, X_tmp, y_train, y_tmp = train_test_split(X, y, test_size=.5,
                                                  random_state=0, stratify=y)
# of remaining 50%, split in half to get 25% validation, 25% test
X_valid, X_test, y_valid, y_test = train_test_split(X_tmp, y_tmp, test_size=.5,
  random_state=0, stratify=y_tmp)

In [ ]:
def find_best_clf(clfs, params, X_train, X_valid, y_train, y_valid, only_best = True):
  #combines clfs and gridsearch params into a dataframe
  models = pd.DataFrame({'clf':clfs, 'params':params})
  #creates a grid search of each clf with associated params
  models['gridsearch'] = models.apply(lambda x: GridSearchCV(x['clf'], x['params'], n_jobs =-1, refit=True), 
                                      axis = 1)
  #fits each grid search to training data
  models.apply(lambda x: x['gridsearch'].fit(X_train, y_train), axis=1)
  #finds accuracy score of each clf on validation data
  models['accuracy'] = models.apply(lambda x: x['gridsearch'].score(X_valid, y_valid), axis = 1)
  #sorts models by accuracy
  models.sort_values('accuracy', ascending=False, inplace=True)
  #returns estimator with highest validation accuracy if only_best is true
  #else returns dataframe of all gridsearchs and acurracies sorted by accuracy
  return models.iloc[0].gridsearch.best_estimator_ if only_best else models[['gridsearch','accuracy']]

In [ ]:
clf = [#DecisionTreeClassifier(random_state=0),
       Pipeline(steps=[('pca', PCA()), ('scaler', StandardScaler()), ('knn', KNeighborsClassifier())])
      ]
params = [
        #{
        #    'max_depth': np.arange(1,20,2), 
        #    'min_samples_split': np.arange(2,20,2),
        #    'min_samples_leaf': np.arange(1,20,2)
        #},
        {
            'pca__n_components': np.arange(1,10),
            'knn__n_neighbors': np.arange(1,20,2),
            'knn__leaf_size': np.arange(1,20,2),
            'knn__p':[1,2]
        }
         ]
grid_search = find_best_clf(clf, params, X_train, X_valid, y_train, y_valid)

In [ ]:
grid_search.score(X_test, y_test)

Regression on price

In [4]:
X = df.drop(columns='price')
y = df.price
# split 80% training data, 20% "_tmp" for validation & test
X_train, X_tmp, y_train, y_tmp = train_test_split(X, y, test_size=.2,
                                                  random_state=0)
# of remaining 20%, split in half to get 10% validation, 10% test
X_valid, X_test, y_valid, y_test = train_test_split(X_tmp, y_tmp, test_size=.5,
  random_state=0)

In [7]:
# I think we should select the main variables instead of using all of them
# also the scores and MSEs are crazy
X.corr()

,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom
squareMeters,1.00e+00,9.57e-03,-6.65e-03,-5.59e-03,1.11e-03,-1.54e-03,8.76e-03,1.66e-02,-7.21e-03,-1.07e-02,7.48e-03,-3.96e-03,-5.88e-04,-1.72e-02,-3.49e-03,-6.23e-04
numberOfRooms,9.57e-03,1.00e+00,-1.12e-02,1.70e-02,2.22e-02,9.04e-03,8.34e-03,1.68e-02,3.98e-03,-2.86e-03,-1.66e-03,-1.40e-02,1.21e-02,2.32e-02,-4.76e-03,-1.55e-02
hasYard,-6.65e-03,-1.12e-02,1.00e+00,1.55e-02,-8.83e-04,6.76e-03,5.02e-03,4.28e-03,2.21e-03,-8.37e-03,-7.60e-03,-8.56e-03,-3.08e-03,-4.63e-03,-9.51e-03,-7.28e-03
hasPool,-5.59e-03,1.70e-02,1.55e-02,1.00e+00,-4.01e-03,8.07e-03,1.46e-02,-6.85e-03,1.89e-03,1.88e-04,-1.00e-03,-7.27e-03,-1.19e-02,4.83e-03,1.24e-03,1.12e-03
floors,1.11e-03,2.22e-02,-8.83e-04,-4.01e-03,1.00e+00,2.21e-03,-4.92e-03,2.46e-03,5.02e-03,2.46e-03,-8.57e-03,6.23e-03,-2.70e-04,1.13e-02,3.62e-03,-2.12e-02
cityCode,-1.54e-03,9.04e-03,6.76e-03,8.07e-03,2.21e-03,1.00e+00,1.13e-02,-7.55e-03,9.27e-03,-2.24e-04,-4.94e-03,2.65e-03,-2.02e-03,-2.21e-03,2.55e-03,-3.34e-03
cityPartRange,8.76e-03,8.34e-03,5.02e-03,1.46e-02,-4.92e-03,1.13e-02,1.00e+00,9.24e-03,7.75e-03,-1.87e-03,5.22e-03,4.74e-03,1.07e-02,-1.65e-03,-1.13e-02,-7.15e-03
numPrevOwners,1.66e-02,1.68e-02,4.28e-03,-6.85e-03,2.46e-03,-7.55e-03,9.24e-03,1.00e+00,6.86e-03,-1.74e-02,2.52e-03,-8.62e-04,7.19e-04,2.03e-02,3.17e-02,-6.08e-03
made,-7.21e-03,3.98e-03,2.21e-03,1.89e-03,5.02e-03,9.27e-03,7.75e-03,6.86e-03,1.00e+00,-1.68e-03,-6.45e-04,-5.51e-03,1.38e-02,5.69e-03,-7.87e-03,-5.43e-03
isNewBuilt,-1.07e-02,-2.86e-03,-8.37e-03,1.88e-04,2.46e-03,-2.24e-04,-1.87e-03,-1.74e-02,-1.68e-03,1.00e+00,3.20e-03,-1.60e-02,2.01e-02,2.75e-03,7.01e-03,1.99e-02


In [8]:
#OLS regression
model = linear_model.LinearRegression()
model.fit(X_train, y_train)

score_train = model.score(X_train, y_train)
score_test = model.score(X_test, y_test)

MSE_train = (1/y_train.size) * np.sum((y_train - model.predict(X_train))**2)
MSE_test = (1/y_test.size)  * np.sum((y_test - model.predict(X_test))**2)

print(f'training score = {score_train}\ntesting score = {score_test}')
print(f'MSE train = {MSE_train}\nMSE test = {MSE_test}')

training score = 0.9999995647424506
testing score = 0.9999995640450483
MSE train = 3628693.3065122473
MSE test = 3579048.2974175224


In [9]:
#kNN regression
model = KNeighborsRegressor(n_neighbors=5, metric='euclidean')
model.fit(X_train, y_train)

score_train = model.score(X_train, y_train)
score_test = model.score(X_test, y_test)

MSE_train = (1/y_train.size) * np.sum((y_train - model.predict(X_train))**2)
MSE_test = (1/y_test.size)  * np.sum((y_test - model.predict(X_test))**2)

print(f'training score = {score_train}\ntesting score = {score_test}')
print(f'MSE train = {MSE_train}\nMSE test = {MSE_test}')

training score = 0.9995415967132242
testing score = 0.9993693826862334
MSE train = 3821656719.1349316
MSE test = 5177162948.909237


In [15]:
#lasso regression
model = linear_model.Lasso(max_iter=2000)
model.fit(X_train, y_train)

score_train = model.score(X_train, y_train)
score_test = model.score(X_test, y_test)

MSE_train = (1/y_train.size) * np.sum((y_train - model.predict(X_train))**2)
MSE_test = (1/y_test.size)  * np.sum((y_test - model.predict(X_test))**2)

print(f'training score = {score_train}\ntesting score = {score_test}')
print(f'MSE train = {MSE_train}\nMSE test = {MSE_test}')
print(f'coefficients = {model.coef_}')

training score = 0.9999995647400114
testing score = 0.9999995640294094
MSE train = 3628713.6412790245
MSE test = 3579176.6880394295
coefficients = [ 1.00000164e+02 -5.70522368e-02  3.00934289e+03  2.98187141e+03
  5.43680462e+01 -8.43634974e-04  5.33938599e+01 -2.97155614e+00
 -1.11867029e+00  1.08817727e+02  1.24877209e+02 -1.75901789e-04
 -8.08140944e-03  1.14279869e-01  5.90596549e+00 -9.14783933e+00]


In [14]:
#ridge regression
model = linear_model.Ridge()
model.fit(X_train, y_train)

score_train = model.score(X_train, y_train)
score_test = model.score(X_test, y_test)

MSE_train = (1/y_train.size) * np.sum((y_train - model.predict(X_train))**2)
MSE_test = (1/y_test.size)  * np.sum((y_test - model.predict(X_test))**2)

print(f'training score = {score_train}\ntesting score = {score_test}')
print(f'MSE train = {MSE_train}\nMSE test = {MSE_test}')
print(f'coefficients = {model.coef_}')

training score = 0.9999995647423179
testing score = 0.9999995640571346
MSE train = 3628694.4123696517
MSE test = 3578949.073229415
coefficients = [ 1.00000165e+02 -5.74626438e-02  3.01184252e+03  2.98427544e+03
  5.43697416e+01 -8.44035948e-04  5.35120030e+01 -3.11392225e+00
 -1.13161782e+00  1.12765900e+02  1.28809549e+02 -1.50176586e-04
 -8.09188207e-03  1.14080012e-01  9.99553728e+00 -9.26475732e+00]


In [ ]:
#kernel regression